In [49]:
import tensorflow as tf
import os
import cv2
import random
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt



In [50]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [51]:
base_dir='/content/drive/MyDrive/Colab Notebooks/test_set/'

In [52]:
#preprocessing

In [53]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

train_datagen= tf.keras.preprocessing.image.ImageDataGenerator(
    
    rescale = 1./255,
    zoom_range = 0.2,
    horizontal_flip = True,
    validation_split = 0.1)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.1 
)



In [54]:
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset='training'

)
validation_generator = validation_datagen.flow_from_directory(
    base_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

Found 1821 images belonging to 2 classes.
Found 202 images belonging to 2 classes.


In [55]:
from tensorflow.keras.layers import Input,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from glob import glob

In [56]:
IMAGE_SIZE=[224,224]
vgg= VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [57]:
for layer in vgg.layers:
  layer.trainable=False

In [58]:
folders=glob('/content/drive/MyDrive/Colab Notebooks/test_set/*')
print(len(folders))

2


In [60]:
x=Flatten()(vgg.output)
prediction= Dense(len(folders),activation='softmax')(x)
model= Model(inputs=vgg.input,outputs=prediction)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [63]:
model.compile(loss='binary_crossentropy', optimizer ='adam', metrics= ['accuracy'])


In [64]:
epoch =10

history= model.fit(train_generator,
                   steps_per_epoch= len(train_generator),
                   epochs= epoch,
                   validation_data =validation_generator,
                   validation_steps= len(validation_generator)
                   )




Epoch 1/10
29/29 [==============================] - 1087s 37s/step - loss: 0.5675 - accuracy: 0.7479 - val_loss: 0.3534 - val_accuracy: 0.8713
Epoch 2/10
29/29 [==============================] - 1050s 36s/step - loss: 0.3085 - accuracy: 0.8753 - val_loss: 0.2835 - val_accuracy: 0.8812
Epoch 3/10
29/29 [==============================] - 1043s 36s/step - loss: 0.2412 - accuracy: 0.9072 - val_loss: 0.2589 - val_accuracy: 0.8911
Epoch 4/10
29/29 [==============================] - 1044s 36s/step - loss: 0.2119 - accuracy: 0.9237 - val_loss: 0.2584 - val_accuracy: 0.8861
Epoch 5/10
29/29 [==============================] - 1043s 36s/step - loss: 0.1849 - accuracy: 0.9352 - val_loss: 0.2759 - val_accuracy: 0.8812
Epoch 6/10
29/29 [==============================] - 1059s 37s/step - loss: 0.1730 - accuracy: 0.9396 - val_loss: 0.2292 - val_accuracy: 0.8762
Epoch 7/10
29/29 [==============================] - 1074s 37s/step - loss: 0.1510 - accuracy: 0.9539 - val_loss: 0.2258 - val_accuracy: 0.8861

In [ ]:
#prediction

In [85]:
from keras.preprocessing import image


img_pred=image.load_img('/content/drive/MyDrive/Colab Notebooks/test_set/dogs/dog.4958.jpg',target_size=(224,224))
img_pred=image.img_to_array(img_pred)
img_pred=np.expand_dims(img_pred, axis=0)

result= model.predict(img_pred)

print(result)
if result[0][0]>result[0][1]:
  prediction="cat"
else:
    prediction="dog"
    print(prediction)


[[0. 1.]]
dog
